In [1]:
import requests
import json
import urllib2
import pandas as pd
import numpy as np

In [2]:
clientid = '6c69ac031e1d4322b3d4cc70af419daf'
access_token = '707017722.6c69ac0.4d57cec659004efab15c099f98fdfe3e'
client_secret = "0bc2259928134b82b76fba0f3f044e0c"
brandnames = ['burberry', 'hm', 'nike', 'macys', 'nordstrom', 'ralphlauren', 'underarmour', 'coach', 'michaelkors','americaneagle','tommyhilfiger','levis','oldnavy','urbanoutfitters','jcrew','abercrombie','jcpenney','anthropologie','topshop', 'gap']
count = 33

In [3]:
# Getting User ID


def userid(name, clientid):
    user_id_api = 'https://api.instagram.com/v1/users/search?q=' + name + '&client_id=' + clientid
    request = requests.get(user_id_api)
    data = request.json()
    for item in data['data']: 
        return item['id']

In [4]:

def userdetails(user_id):
    api = 'https://api.instagram.com/v1/users/' + str(user_id) + '/?access_token=' + access_token
    request = requests.get(api)
    data = request.json()
    return {'followed_by': data['data']['counts']['followed_by'], 'media': data['data']['counts']['media']}
    

In [5]:

def followedby(user_id):
    api = 'https://api.instagram.com/v1/users/' + str(user_id) + '/?access_token=' + access_token
    request = requests.get(api)
    data = request.json()
    return {'followed_by': data['data']['counts']['followed_by']}

In [6]:
#get data from first api request

def getfirstdata(access_token, user_id):
    url = 'https://api.instagram.com/v1/users/' + str(user_id) + '/media/recent/?access_token=' + access_token + '&count=' + str(count)
    request = requests.get(url)
    data = request.json()
    return data

    
    

    

In [7]:
#function to extract data from json

def getdataframe(data, _id):

    
    username = []
    user_id =[]
    link =[]
    normallink =[]
    tags =[]
    text =[]
    comments =[]
    created_time = []
    img_or_vid = []
    likes =[]
    users_in_photo = []
    lat = []
    lon = []
    loc = []
    
   
    
    for item in data['data']:
        
        try:
            username.append(item['caption']['from']['username'])
        except (KeyError, ValueError, TypeError) as e:
            username.append(np.NaN)
        
        try:
            user_id.append(item['caption']['from']['id'])
        except (KeyError, ValueError, TypeError) as e:
            user_id.append(np.NaN)
            
        try:
            link.append(item['images']['standard_resolution']['url'])
        except (KeyError, ValueError, TypeError) as e:    
            link.append(np.NaN)
        
        try:
            tags.append(item['tags'])
        except (KeyError, ValueError, TypeError) as e:    
            tags.append(np.NaN)
        
        try:
            text.append(item['caption']['text'])
        except (KeyError, ValueError, TypeError) as e: 
            text.append(np.NaN)
            
        
        try:
            comments.append(item['comments']['count']) 
        except (KeyError, ValueError, TypeError) as e:   
            comments.append(np.NaN)
        
        try:
            created_time.append(item['created_time'])
        except (KeyError, ValueError, TypeError) as e:
            created_time.append(np.NaN)
        
        try:
            img_or_vid.append(item['type']) 
        except (KeyError, ValueError, TypeError) as e: 
            img_or_vid.append(np.NaN)
        
        try:
            likes.append(item['likes']['count'])
        except (KeyError, ValueError, TypeError) as e:    
            likes.append(np.NaN)
        
        try:
            lat.append(item['location']['latitude'])
        except (KeyError, ValueError, TypeError) as e:
            lat.append(np.NaN)
        
        try:
            lon.append(item['location']['longitude'])
        except (KeyError, ValueError, TypeError) as e:
            lon.append(np.NaN)
        
        try:
            loc.append(item['location']['name'])
        except (KeyError, ValueError, TypeError) as e:
            loc.append(np.NaN)
               
        try:
            normallink.append(item['link'])
        except (KeyError, ValueError, TypeError) as e:
            loc.append(np.NaN)
            
            
    users_in_photo = []
    try:
        for item in data['data']:
            templist = []
            for item in item['users_in_photo']:
                templist.append(item['user']['id'])
            users_in_photo.append(len(templist))
    except (KeyError, ValueError, TypeError) as e:
        users_in_photo.append(np.NaN)
    
    
    mediatotal = userdetails(_id)['media']
    media = [mediatotal for i in range(len(username))]
              
    raw_data = {'username':username, 'likes':likes, 'comments':comments, 'img_or_vid':img_or_vid, 'link':link,
        'tags':tags,'users_in_photo':users_in_photo, 'created_time':created_time, 'text':text, 'lat':lat, 'lon':lon, 'loc':loc, 'media':media, 'normallink':normallink, 
        }
    
    dftemp = pd.DataFrame(raw_data, columns = ['username', 'likes', 'comments', 'img_or_vid', 'link', 'tags','users_in_photo', 'created_time', 'text', 'lat', 'lon', 'loc','media','normallink'])
    return dftemp
    

In [ ]:
# function to extract data from multiple pages of instagram accoun

def pagination(_id):
    pagination_link = data['pagination']['next_url']    
    request = requests.get(pagination_link)
    global data
    data = request.json()
    dftemp = getdataframe(data, _id)
    updated_df = pd.concat([df, dftemp], axis=0)
    return updated_df

    

In [8]:
#create a list of the id's of instagram accounts

brandnames_ids = []
for name in brandnames:
    brandnames_ids.append(userid(name, clientid))




<ipython-input-8-8f70a7e6a58f>:4: SyntaxWarning: name 'data' is used prior to global declaration
  global data


In [ ]:
#Create Dataframe
df = pd.DataFrame()




In [9]:
#Loop through all names to get all but first 300 images from each account

for id_ in brandnames_ids[:1]:
    data = getfirstdata(access_token, id_)
    dftemp = getdataframe(data, id_)
    df = pd.concat([df, dftemp], axis=0)
    for i in range( (userdetails(id_)['media']-300)/33):
        df = pagination(id_)
    

In [15]:
df = df.reset_index(drop=True)

In [ ]:
#Save dataframe to csv

df.to_csv("file.csv", encoding="utf-8") 

In [6]:
brandnames = ['burberry', 'hm', 'nike', 'macys', 'nordstrom', 'ralphlauren', 'underarmour', 'coach', 'michaelkors','americaneagle','tommyhilfiger','levis','oldnavy','urbanoutfitters','jcrew','abercrombie','jcpenney','anthropologie','topshop', 'gap']

user = [userid(item, clientid) for item in brandnames]

followed_by = [followedby(i).get('followed_by') for i in user]

In [52]:

pd.DataFrame({'username':brandnames, 'id':user, 'followed_by':followed_by}).to_csv('followed_by.csv')


/Users/Jay/Desktop/Portfolioproject


5207797

[u'1889387',
 u'23410080',
 u'13460080',
 u'13806065',
 u'25501780',
 u'10291533',
 u'5858873',
 u'9791606',
 u'1906837',
 u'1935199',
 u'8579765',
 u'4641204',
 u'214412757',
 u'5413187',
 u'183814919',
 u'23788300',
 u'256767875',
 u'4163308',
 u'4165329',
 u'9456013']

In [ ]:
#import urllib
#from instagram.client import InstagramAPI



#count = 20

#def downloadimages(access_token, client_secret, count)

#api = InstagramAPI(access_token=access_token, client_secret=client_secret)
#recent_media, next_ = api.user_recent_media(user_id=user_id, count=count)
#photos = []
#for item in recent_media:
    #print item.likes
    #print item.get_standard_resolution_url()
    
#for item in recent_media:
#    print item.comments[1]
    #print media.likes
    
    #photos.append('<img src="%s"/>' % item.get_standard_resolution_url())
    #urllib.urlretrieve(media.get_standard_resolution_url(), `count`+".jpg")
    #count = count + 1
    #content += ''.join(photos)



#for media in recent_media:
    #print media.caption

In [ ]:
#import urllib
##from instagram.client import InstagramAPI

#content = "<h2>Tag Search</h2>"
#if not access_token:
#    print 'Missing Access Token'
#try:
#    api = InstagramAPI(access_token=access_token)
#    tag_search, next_tag = api.tag_search(q="selfie")
#    tag_recent_media, next = api.tag_recent_media(tag_name=tag_search[0].name)
#    photos = []
#    count = 0
#    for tag_media in tag_recent_media:
#        photos.append('<img src="%s"/>' % tag_media.get_standard_resolution_url())
#        urllib.urlretrieve(tag_media.get_standard_resolution_url(), `count`+".jpg")
#        count = count + 1
#    content += ''.join(photos)
#except Exception, e:
#    print e   

In [3]:
import pandas as pd
df = pd.read_csv('df.csv')

In [31]:
import urllib

for row in range(14593,len(df['link'])):
    url = df['link'][row]
    filename = df['link'][row].split("/")[-1]
    urllib.urlretrieve(url, "/Users/Jay/Desktop/Portfolioproject/pics/%s.jpg" % (filename))
    print row
    


14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
14614
14615
14616
14617
14618
14619
14620
14621
14622
14623
14624
14625
14626
14627
14628
14629
14630
14631
14632
14633
14634
14635
14636
14637
14638
14639
14640
14641
14642
14643
14644
14645
14646
14647
14648
14649
14650
14651
14652
14653
14654
14655
14656
14657
14658
14659
14660
14661
14662
14663
14664
14665
14666
14667
14668
14669
14670
14671
14672
14673
14674
14675
14676
14677
14678
14679
14680
14681
14682
14683
14684
14685
14686
14687
14688
14689
14690
14691
14692
14693
14694
14695
14696
14697
14698
14699
14700
14701
14702
14703
14704
14705
14706
14707
14708
14709
14710
14711
14712
14713
14714
14715
14716
14717
14718
14719
14720
14721
14722
14723
14724
14725
14726
14727
14728
14729
14730
14731
14732
14733
14734
14735
14736
14737
14738
14739
14740
14741
14742
14743
14744
14745
14746
14747
14748
14749
14750
14751
14752
14753
14754
14755
14756
14757
14758
1475

In [ ]:
9907

In [ ]:
dir1

In [2]:
!pwd

/Users/Jay/Desktop/Portfolioproject


In [5]:
import os
os.listdir("/Users/Jay/Desktop/Portfolioproject/pics")[1]


'10004085_481371781990656_1654681798_n.jpg.jpg'

In [6]:
import pandas as pd
test = pd.DataFrame()

""


In [ ]:
caffe.set_mode_gpu()
os.chdir('/home/ubuntu/pics/')

i = 0
for item in jpeg_list: 
    try:
        input_image = caffe.io.load_image('/home/ubuntu/pics/' + item) 
        prediction = net.predict([input_image], oversample=False)
        jpeg_class[i] = prediction[0].argmax()
        print i 
        i+=1
    except:
        jpeg_class[i] = np.nan
        print item
        print i
        i+=1
    
import copy
jpeg_class2 = copy.copy(jpeg_class)

import numpy as np

for n,i in enumerate(jpeg_class2):
    if i=='e':
...      jpeg_class[n]=np.nan
    
    

In [22]:
tag_name = 'whocares'
url_tag_api = 'https://api.instagram.com/v1/tags/' + tag_name  + '?access_token=' + access_token

request = requests.get(url_tag_api)
data = request.json()


In [26]:
url_tag_api = 'https://api.instagram.com/v1/tags/' + tag_name  + '/media/recent' + ?access_token=ACCESS-TOKEN

SyntaxError: invalid syntax (<ipython-input-26-e1a3ea816229>, line 1)

In [27]:
'https://api.instagram.com/v1/tags/' + tag_name  + '/media/recent' + ?access_token=ACCESS-TOKEN

SyntaxError: invalid syntax (<ipython-input-27-7adbbbff1f0e>, line 1)

In [ ]:
url_tag_api = 'https://api.instagram.com/v1/tags/' + tag_name  + '?access_token=' + access_token